In [ ]:
from jesse import jesse
jesse.SetOptions()
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pybaseball as mlb
import mlbgame

import preprocess
from preprocess import service, model, dates
from process import batter_processing


/Users/downing.jesse/Projects/MLB_statTRAX/python
FIRST-TIME-USE INITIALIZE: pulling batter data from pybaseball api.  This will take several minutes...


### Test process

In [12]:
import os as _os
def build_path(file_name='raw_batter_stats'):
    file_name = file_name
    path = None
    if _os.name == 'nt':
        data_dir = '..\\data\\csv\\'
    elif _os.name == 'posix':
        data_dir = '../data/csv/'
    return data_dir+file_name+'.csv'
def save(raw_stats_df):
    raw_stats_df.to_csv(path)


In [13]:
build_path()

'../data/csv/raw_batter_stats.csv'

In [38]:
read_obj = model.ReadData()
read_obj.raw_stats_df

In [42]:
 #preprocess.get_raw_batter_stats(year=preprocess.get_prior_years())
read_obj.set_raw_stats_df() 

pulling csv
(225069, 37)


In [86]:
read_obj.update_raw_stats_df()

dates we dont got ['2019-05-31', '2019-06-01', '2019-06-02']
Updating batter data to most recent date...
 Adding stats from dates...
['2019-05-31', '2019-06-01', '2019-06-02']
updated raw_stats_df in model.py
(226024, 37)


/Users/downing.jesse/Projects/MLB_statTRAX/python/preprocess/model.py:51: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.raw_stats_df = _pd.concat([self.raw_stats_df, new_df])


In [82]:
#read_obj.update_raw_stats_df()
import pybaseball as _pybaseball
#dates_we_dont_got = ['2019-05-31', '2019-06-01', '2019-06-02']
#new_df = _pd.DataFrame(preprocess.service.get_raw_stats(dates_we_dont_got))

def get_raw_stats(season_days_so_far):
        df_lst = []
        for d in season_days_so_far:
            print(d)
            try:
                tmp_df = _pybaseball.batting_stats_range(str(d), )
                tmp_df['DATE'] = d
                df_lst.append(tmp_df)
            except:
                continue
        new_df = pd.concat(df_lst)
        return new_df

In [83]:
df = get_raw_stats(['2019-05-31', '2019-06-01', '2019-06-02'])
df.head()

2019-05-31
2019-06-01
2019-06-02


,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,DATE
1,Jose Abreu,32,2,MLB-AL,Chicago,1,5,4,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0.000,0.0,0.000,0.000,2019-05-31
2,Ronald Acuna Jr.,21,2,MLB-NL,Atlanta,1,4,4,1,2,0,0,0,0,0,0,1,0,0,0,0,1,0,0.500,0.5,0.500,1.000,2019-05-31
3,Willy Adames,23,2,MLB-AL,Tampa Bay,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0.500,0.5,0.500,1.000,2019-05-31
4,Matt Adams,30,2,MLB-NL,Washington,1,4,4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.000,0.0,0.000,0.000,2019-05-31
5,Jesus Aguilar,29,2,MLB-NL,Milwaukee,1,4,3,1,1,0,0,1,1,1,0,2,0,0,0,0,0,0,0.333,0.5,1.333,1.833,2019-05-31


In [37]:
#dates = preprocess.get_mlb_dates()
# def get_mlb_dates(year=None):
#         obj = dates.GetSchedule(year)
#         obj.set_prior_years()
#         obj.set_mlb_dates()
#         obj.set_endpoints()
#         obj.set_all_star_break_days()
#         obj.set_season_days_so_far()
#         return obj

Updating batter data to most recent date...
['2019-05-31', '2019-06-01', '2019-06-02']


,0
0,Name Age #days Lev...
1,Name Age #days Lev...
2,Name Age #days Lev...


In [ ]:

save_obj = model.StoreData()
save_obj.save(read_obj.get_raw_stats_df())

In [15]:
read_obj.get_raw_stats_df()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,DATE
0,0,0,0,0,0,0,0,0,0,Matt Adams,26,1516,MLB-NL,St. Louis,1,4,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0.000,0.000,0.000,0.000,2015-04-05
1,21,5,5,16,4,17,2,20,8,Matt Holliday,35,1516,MLB-NL,St. Louis,1,5,4,0,2,0,0,0,2,1,0,1,0,0,0,0,1,0,0.500,0.600,0.500,1.100,2015-04-05
2,20,8,8,13,10,11,18,6,15,Jhonny Peralta,33,1516,MLB-NL,St. Louis,1,5,5,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0.200,0.200,0.200,0.400,2015-04-05
3,19,7,7,14,1,21,5,16,4,Starlin Castro,25,1516,MLB-NL,Chicago,1,4,4,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.250,0.250,0.250,0.500,2015-04-05
4,17,2,2,20,8,13,10,11,18,David Ross,38,1516,MLB-NL,Chicago,1,2,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.500,0.500,1.000,1.500,2015-04-05
5,16,4,4,17,2,20,8,13,10,Tommy La Stella,26,1516,MLB-NL,Chicago,1,3,3,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.333,0.333,0.333,0.667,2015-04-05
6,15,3,3,19,7,14,1,21,5,Chris Coghlan,30,1516,MLB-NL,Chicago,1,4,4,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0.250,0.250,0.500,0.750,2015-04-05
7,14,1,1,21,5,16,4,17,2,Peter Bourjos,28,1516,MLB-NL,St. Louis,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,2015-04-05
8,13,10,10,11,18,6,15,3,19,Jorge Soler,23,1516,MLB-NL,Chicago,1,4,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000,0.000,0.000,0.000,2015-04-05
9,12,9,9,12,9,12,9,12,9,Jon Jay,30,1516,MLB-NL,St. Louis,1,4,4,0,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0.500,0.500,0.750,1.250,2015-04-05


In [2]:
# this was build by preprocess
batter_data = pd.read_csv('../data/csv/raw_batter_stats.csv')

In [3]:
batter_data 

,Unnamed: 0,#days,2B,3B,AB,Age,BA,BB,CS,DATE,G,GDP,H,HBP,HR,IBB,Lev,Name,OBP,OPS,PA,R,RBI,SB,SF,SH,SLG,SO,Tm,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1
0,0,1516,0,0,4,26,0.000,0,0,2015-04-05,1,0,0,0,0,0,MLB-NL,Matt Adams,0.000,0.000,4,0,0,0,0,0,0.000,2,St. Louis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21,1516,0,0,2,31,0.000,0,0,2015-04-05,1,0,0,0,0,0,MLB-NL,Miguel Montero,0.000,0.000,2,0,0,0,0,0,0.000,1,Chicago,5.0,16.0,16.0,4.0,17.0,2.0,20.0,8.0,13.0
2,20,1516,0,0,5,29,0.400,0,1,2015-04-05,1,0,2,0,0,0,MLB-NL,Matt Carpenter,0.400,0.800,5,1,1,0,0,0,0.400,0,St. Louis,8.0,13.0,13.0,10.0,11.0,18.0,6.0,15.0,3.0
3,19,1516,0,0,3,25,0.000,1,0,2015-04-05,1,0,0,0,0,0,MLB-NL,Anthony Rizzo,0.250,0.250,4,0,0,0,0,0,0.000,2,Chicago,7.0,14.0,14.0,1.0,21.0,5.0,16.0,4.0,17.0
4,17,1516,1,0,4,29,0.250,0,0,2015-04-05,1,0,1,0,0,0,MLB-NL,Dexter Fowler,0.250,0.750,4,0,0,0,0,0,0.500,2,Chicago,2.0,20.0,20.0,8.0,13.0,10.0,11.0,18.0,6.0
5,16,1516,0,0,1,31,0.000,0,0,2015-04-05,1,0,0,0,0,0,MLB-NL,Jon Lester,0.000,0.000,1,0,0,0,0,0,0.000,1,Chicago,4.0,17.0,17.0,2.0,20.0,8.0,13.0,10.0,11.0
6,15,1516,0,0,1,31,0.000,0,0,2015-04-05,1,0,0,0,0,0,MLB-NL,Mark Reynolds,0.000,0.000,1,0,0,0,0,0,0.000,1,St. Louis,3.0,19.0,19.0,7.0,14.0,1.0,21.0,5.0,16.0
7,14,1516,0,0,3,33,0.000,0,0,2015-04-05,1,0,0,0,0,0,MLB-NL,Adam Wainwright,0.000,0.000,3,0,0,0,0,0,0.000,1,St. Louis,1.0,21.0,21.0,5.0,16.0,4.0,17.0,2.0,20.0
8,13,1516,2,0,5,25,0.600,0,0,2015-04-05,1,0,3,0,0,0,MLB-NL,Jason Heyward,0.600,1.600,5,1,0,1,0,0,1.000,0,St. Louis,10.0,11.0,11.0,18.0,6.0,15.0,3.0,19.0,7.0
9,12,1516,0,0,2,32,0.000,2,0,2015-04-05,1,0,0,0,0,1,MLB-NL,Yadier Molina,0.500,0.500,4,0,0,0,0,0,0.000,2,St. Louis,9.0,12.0,12.0,9.0,12.0,9.0,12.0,9.0,12.0


In [ ]:
cln_batter_obj = batter_processing.RawBatter(batter_data)
cln_batter_obj.process_batter()
cln_batter = cln_batter_obj.get_processed_batter()

In [ ]:
cln_batter.head()

In [ ]:
# Look for a file
# If it's there, list all its dates
# Pull a list of all the dates I want

In [ ]:
test.Name.value_counts()

In [ ]:
class ReadData:
    def __init__(self):
        self.raw_stats_df = None
    def set_raw_stats_df(self):
        try:
            self.raw_stats_df = _pd.read_csv('../data/csv/raw_batter_stats.csv')
            print('Pulling raw batter stats from existing file...')
        except OSError:
            print('FIRST-TIME-USE INITIALIZE: pulling batter data from pybaseball api...')
            self.raw_stats_df = preprocess.get_raw_batter_stats(year=preprocess.get_prior_years())
    def update_dates_in_raw_stats_df(self):
        dates = preprocess.get_mlb_dates()
        season_days_so_far = dates.get_season_days_so_far()
        dates_we_got = list(self.raw_stats_df.DATE)
        dates_we_dont_got = [d for d in season_days_so_far if d not in dates_we_got]
        if len(dates_we_dont_got) > 0:
            print('Updating batter data to most recent date...')
            print(dates_we_dont_got)
            new_df = _pd.DataFrame(preprocess.service.get_raw_stats(dates_we_dont_got))
            self.raw_stats_df = _pd.concat([self.raw_stats_df, new_df])
        else: print('Batter data is current.')
        self.raw_stats_df.drop_duplicates(inplace=True)
        self.raw_stats_df.sort_values('DATE', inplace=True)
    def get_raw_stats_df(self):
        return self.raw_stats_df

In [ ]:
load = pd.read_csv('../data/csv/raw_batter_stats.csv')
load.shape

In [ ]:
test.shape

### Aggregate all player stats for 2018

In [ ]:
df = pd.DataFrame()
df_lst = []

for d in season_days_so_far:
    try:
        tmp_df = mlb.batting_stats_range(str(d), )
        print(' processing schedule date ', d)
    except:
        continue
    tmp_df['DATE'] = d
    df_lst.append(tmp_df)

In [ ]:
df = pd.concat(df_lst)

In [ ]:
df.shape

### Get key for a player

In [ ]:
mlb.playerid_lookup('gallo', 'joey')

In [ ]:
id = int(mlb.playerid_lookup('gallo', 'joey')['key_mlbam'])
print(id)

### Get season-level stats for a all batters

In [ ]:
mlb.batting_stats_bref(season=2019).head() # entire season
mlb.batting_stats_range(start_dt='2019-03-20', end_dt=today) # specified range

#data = mlb.batting_stats_range("2016-08-24",) 
data = mlb.batting_stats_range("2019-05-28", ) 


In [ ]:
df = pd.DataFrame()
df_lst = []

for d in season_days_so_far:
    try:
        tmp_df = mlb.batting_stats_range(str(d), )
        print(' processing schedule date ',d)
    except:
        continue
    tmp_df['DATE'] = d
    df_lst.append(tmp_df)

In [ ]:
df = pd.concat(df_lst)

In [ ]:
df.sort_values(['Name', 'DATE'], inplace=True)

In [ ]:
df

In [ ]:
today = datetime.today()
days = [today - datetime.timedelta(days=x) for x in range(0, numdays)]

In [ ]:
from datetime import date, timedelta

d1 = date(2008, 8, 15)  # start date
d2 = date(2008, 9, 15)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    print(d1 + timedelta(days=i))



In [ ]:
# Make a list of all days during 2019 baseball season (from beginning to now)
# For each day, pull all data, and append to master dataframe
# data

## Temp

### Get raw stats for a batter

In [ ]:
#gallo_stats = mlb.statcast_batter('2019-04-01', '2019-05-01', id)
gallo_stats = mlb.statcast_batter(start_dt='2019-02-20', end_dt=today, player_id=id)

In [ ]:
gallo_stats[gallo_stats.game_date == '2019-05-01']

In [ ]:
gallo_stats

### Grab all batted ball events for a single day for a single player

In [ ]:
gallo_stats[(gallo_stats.game_date == '2019-05-01') & (~gallo_stats.events.isnull())]

In [ ]:
gallo_stats.events.value_counts()

## What other player sources exist?
* Lahman
** batting stats by year
** updated at the end of each season (current is through 2018)
* fangraps
* baseball reference
* baseball savants


### Do I have to parse the data to one-record-per-game format myself?

### Can I use retrosheet for past years, and use stat_cast for current year?

### How do I define the first game date of the season?

### I should save all player IDs to a table, and append any new players?
